In [4]:
import requests

# Base URL for Solana RPC endpoint
rpc_url = "https://api.mainnet-beta.solana.com"

# List of wallet addresses holding some tokens (example)
wallet_addresses = [
    'C4su4UzsPgbDApTEWFThKwbTyHE9ZoRnV7TXwrtL55SB',
    'AnsWpQ5KzJpns4fTXQWzWg8YakJMrDS2T6M3TpUX4t5U',
    '93AX7iJUTbzmzB4vHXCRHjfLyrskAfNd1ReRQkfRUnXL',
    # Add more wallet addresses...
]

def get_token_accounts_by_owner(wallet_address):
    headers = {'Content-Type': 'application/json'}
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getTokenAccountsByOwner",
        "params": [
            wallet_address,
            {
                "programId": "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA",
                "encoding": "jsonParsed"
            }
        ]
    }

    response = requests.post(rpc_url, json=payload, headers=headers)
    return response.json()

# Loop through each wallet and retrieve their token accounts
wallet_to_mints = {}

for wallet in wallet_addresses:
    accounts = get_token_accounts_by_owner(wallet)
    # Extract mint addresses for each wallet
    mints = [account['account']['data']['parsed']['info']['mint'] for account in accounts.get('result', {}).get('value', [])]
    
    # Store the mint addresses for each wallet
    wallet_to_mints[wallet] = mints

# Print the mint addresses associated with each wallet
print(wallet_to_mints)

{'C4su4UzsPgbDApTEWFThKwbTyHE9ZoRnV7TXwrtL55SB': [], 'AnsWpQ5KzJpns4fTXQWzWg8YakJMrDS2T6M3TpUX4t5U': [], '93AX7iJUTbzmzB4vHXCRHjfLyrskAfNd1ReRQkfRUnXL': []}
